# 一、读取文本

In [2]:
import pandas as pd
import numpy as np

fake_text = pd.read_csv('DataSet_Misinfo_FAKE.csv').head(1000)
real_text = pd.read_csv('DataSet_Misinfo_TRUE.csv').head(1000)

fake_text.drop(fake_text.columns[0],axis=1,inplace=True)
real_text.drop(real_text.columns[0],axis=1,inplace=True)

In [3]:
mixed_text = pd.concat([real_text,fake_text])['text'].tolist()

for i in range(0,len(mixed_text)):
    mixed_text[i] = str(mixed_text[i])
    
Y = np.concatenate([np.ones(len(real_text)), np.zeros(len(fake_text))])

# 二、采用BERT进行编码

In [4]:
#对文本进行预处理
import torch
import numpy as np
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')#选用large-cased版本

#tokenize
tokenized_text = tokenizer(mixed_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')


#构建TensorDataset用于训练
input_ids = tokenized_text['input_ids']
attention_mask = tokenized_text['attention_mask']
Y = torch.tensor(Y)

data = torch.utils.data.TensorDataset(input_ids,attention_mask,Y)


In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

train_data, test_data = train_test_split(data, test_size=0.2)

train_loaded = DataLoader(train_data, batch_size=32)
test_loaded = DataLoader(test_data, batch_size=32)

In [6]:
from tqdm import tqdm
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-cased')

#定义训练参数
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import os
from torchmetrics.functional import accuracy
from torchmetrics import Recall, F1Score, Precision

os.environ['KMP_DUPLICATE_LIB_OK']='True'

NoneType = type(None)

if torch.cuda.is_available():
    model = model.cuda()
    loss_fn = loss_fn.cuda()

for epoch in range(epochs):
    total_acc_train = 0
    total_loss_train = 0
    model.train()
    
    for batch in tqdm(train_loaded, desc=f"Epoch {epoch + 1}/{epochs}"):
        input_ids, attention_mask, Y = batch
        input_ids = input_ids.squeeze(1).to(device)
        attention_mask = attention_mask.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        output = model(input_ids, attention_mask)
        
        logits = output.logits
        predictions = torch.argmax(logits, dim=1)
        acc = accuracy(predictions, Y,'multiclass', num_classes=2)
        precision =  Precision('multiclass',average='micro', num_classes=2)
        pre = precision(predictions, Y)
        f1score =  F1Score('multiclass', num_classes=2)
        f1 = f1score(predictions, Y)
        recall = Recall('multiclass',average='micro', num_classes=2)
        rec = recall(predictions, Y)
    
        Y1=Y.long()
        loss = loss_fn(logits,Y1)
        loss.backward()
        
        optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():
            for valinput_ids, valattention_mask, valY in test_loaded:
                valinput_ids = valinput_ids.squeeze(1).to(device)
                valattention_mask = valattention_mask.to(device)
                valY = valY.to(device)
        
                optimizer.zero_grad()
                output1 = model(valinput_ids, valattention_mask)
            
                logits1 = output1.logits
                predictions1 = torch.argmax(logits1, dim=1)
                acc1 = accuracy(predictions1, valY,'binary', num_classes=2)
                
                precision =  Precision('multiclass',average='micro', num_classes=2)
                pre1 = precision(predictions1, valY)
                f1score =  F1Score('multiclass', num_classes=2)
                f1_1 = f1score(predictions1, valY)
                recall = Recall('multiclass',average='micro', num_classes=2)
                rec1 = recall(predictions1, valY)
                
                optimizer.step()
                
    print(f'''Epochs: {epoch + 1} 
            | Train Accuracy: {acc: .3f}  
            | Train Precision: {pre: .3f} 
            | Train Recall: {rec: .3f} 
            | Train F1Score: {f1: .3f}
            | Val Accuracy: {acc1: .3f}
            | Val Precision: {pre1: .3f}
            | Val Recall: {rec1: .3f}
            | Val F1Score: {f1_1: .3f}''') 

Epoch 1/3: 100%|██████████| 50/50 [2:38:47<00:00, 190.54s/it]  


Epochs: 1 
            | Train Accuracy:  1.000  
            | Train Precision:  1.000 
            | Train Recall:  1.000 
            | Train F1Score:  1.000
            | Val Accuracy:  1.000
            | Val Precision:  1.000
            | Val Recall:  1.000
            | Val F1Score:  1.000


Epoch 2/3: 100%|██████████| 50/50 [2:38:58<00:00, 190.76s/it]  


Epochs: 2 
            | Train Accuracy:  1.000  
            | Train Precision:  1.000 
            | Train Recall:  1.000 
            | Train F1Score:  1.000
            | Val Accuracy:  1.000
            | Val Precision:  1.000
            | Val Recall:  1.000
            | Val F1Score:  1.000


Epoch 3/3: 100%|██████████| 50/50 [2:37:58<00:00, 189.57s/it]  

Epochs: 3 
            | Train Accuracy:  1.000  
            | Train Precision:  1.000 
            | Train Recall:  1.000 
            | Train F1Score:  1.000
            | Val Accuracy:  1.000
            | Val Precision:  1.000
            | Val Recall:  1.000
            | Val F1Score:  1.000
